## 環境安裝 (切換到原生 Python 3 (ipykernel))
- 更改套件安裝儲存位置 saveFolder=?
- 更改Kernel標籤名稱 label=?
- 更改Image映像檔位置 IMAGE=?

In [ ]:
%%bash
### 0. pipi install libsray savefolder
saveFolder=work
label=LlamaFactory

### 1. IMAGE
IMAGE=/work/u00cjz00/nvidia/cuda118/c00cjz00_cuda11.8_pytorch_2.1.2-cuda11.8-cudnn8-devel-llama_factory.sif
IMAGE_basename=S-${saveFolder}-${label}_$(basename "$IMAGE" .sif)

### 2. VIRTUAL LIBRARY and BINARY FOLDER
libraryFolder=/${saveFolder}/$(whoami)/libraryFolder/${IMAGE_basename}/local/lib
libraryFolder_binding=${libraryFolder}:${HOME}/.local/lib
binFolder=/${saveFolder}/$(whoami)/libraryFolder/${IMAGE_basename}/local/bin
binFolder_binding=${binFolder}:${HOME}/.local/bin
rm -rf /${saveFolder}/$(whoami)/libraryFolder/${IMAGE_basename}
mkdir -p ${libraryFolder} ${binFolder}

# 3. PIP INSTALL SLAVE IPYKERNEL
#ml libs/singularity/3.10.2
singularity exec -B ${libraryFolder_binding} -B ${binFolder_binding} ${IMAGE} pip install -q ipykernel IProgress ipywidgets

# 4. IPYKERNEL for IMAGE
IPYKERNEL=/work/u00cjz00/slurm_jobs/ipykernel/t2/image_with_ipykernel_local
mkdir -p ${HOME}/.local/share/jupyter/kernels/
rm -rf ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}
cp -rf ${IPYKERNEL} ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}
chmod -R 755 ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}
IMAGE_desc=$(echo $IMAGE | sed 's_/_\\/_g')
sed -i "s/templateSIF/${IMAGE_desc}/g" ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}/kernel.json
sed -i "s/templateImage/Image_${IMAGE_basename}/g" ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}/kernel.json
sed -i "s@templateLibrayFolder@${libraryFolder_binding}@g" ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}/kernel.json
sed -i "s@templateBinFolder@${binFolder_binding}@g" ${HOME}/.local/share/jupyter/kernels/${IMAGE_basename}/kernel.json

# 5. check size
du -sh ${libraryFolder}
echo /home/$(whoami)/.local/share/jupyter/kernels/${IMAGE_basename}/kernel.json
cat /home/$(whoami)/.local/share/jupyter/kernels/${IMAGE_basename}/kernel.json

cmd="/work/opt/ohpc/Taiwania3/libs/singularity/3.10.2/bin/singularity exec --nv --cleanenv \
-B /work -B /work/u00cjz00/os/ubuntu/bin:/usr/ubuntu_bin \
-B ${libraryFolder}:/home/g00cjz00/.local/lib \
-B ${binFolder}:/home/g00cjz00/.local/bin \
${IMAGE} \
bash -c 'export PATH=\$PATH:\$HOME/.local/bin; echo \$PATH;'
"

echo ${cmd}

## 環境安裝 (切換到新創建kernel)

In [ ]:
# GPU確認
!nvidia-smi
import torch
torch.cuda.is_available()

In [ ]:
# torch and cuda version
import torch
print(torch.__version__)
print(torch.version.cuda)